In [23]:
import pandas as pd
import os
import numpy as np
import scipy as sp
import sklearn.ensemble as ske
from sklearn.cross_validation import cross_val_score
import tensorflow as tf
from __future__ import print_function
import inspect

# Prelim Analysis

In [24]:
os.chdir('./Kaggle_Digit_Recognizer/')

In [25]:
train = pd.read_csv('train.csv')

In [26]:
test = pd.read_csv('test.csv')

In [27]:
Results=pd.DataFrame(data={'ImageId':test.index+1})

In [28]:
train['label'] = train['label'].astype('category')

In [29]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Simplify By changing images from 0's and 255's to 0's and 1's

In [30]:
train.ix[:,1:]=train.ix[:,1:]/255.

D:\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [31]:
max(train.ix[1,:].as_matrix())

1.0

In [32]:
train.shape

(42000, 785)

In [33]:
test.ix[:,:]=test.ix[:,:]/255.

D:\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [34]:
max(test.ix[1,:].as_matrix())

1.0

In [35]:
test.shape

(28000, 784)

# Random Forest Classifier

In [36]:
clf=ske.RandomForestClassifier(n_estimators=100)

In [37]:
scores = cross_val_score(clf, train.drop('label',axis=1), train['label'])

In [38]:
scores.mean()

0.96290506306874779

In [39]:
clf = clf.fit(train.drop('label',axis=1), train['label'])

In [41]:
Results['Label']=clf.predict(test)

In [42]:
Results.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [43]:
Results[['ImageId','Label']].to_csv('RFpred.csv',index=False)

In [44]:
Results.rename(columns={'Label': 'RFpred'}, inplace=True)

Test Predictions were 96.457% Correct

# Genetic Algorithm Baseline Functions

In [46]:
np.random.normal(size=[1,2])

array([[-0.14770604, -0.26183749]])

In [74]:
np.array([[1,1]]).shape

(1, 2)

In [126]:
simple_reproduce([np.array([1,2])])

array([[  2,  13],
       [ 15,   9],
       [ -4,  -5],
       [ 31,   1],
       [  6, -10]])

In [290]:
#cnv_1,cnv_2,output_1,output_2_scale
def random_population(shape):
    return np.array([np.array([3,3,64,2]),1])
    #return np.random.normal(scale=10.0,size=shape)
# Takes in a population and runs the neural network
# prereq random_population
# takes in a poulation(array of hyperparameters) and a neural network training function

def perturb(l,noise=1.0):
    return l+np.array([1,1,8,1])*np.random.normal(scale=noise,size=l.shape).astype(int)

def simple_reproduce(fitness_top,num_child=10):
    winner=fitness_top[0]
    children=[]
    while len(children)<num_child:
        new_child=perturb(winner)
        if not any(all(e == new_child) for e in children):
            children.append(new_child)
    return np.array(children).astype(int)

def test_individuals(pop,nn_alg):
    fit=[]
    for per in pop:
        fitness=nn_alg(per)
        fit.append([per,fitness])
    fit_sorted=sorted(fit,key=lambda x: x[1],reverse=True)
    return fit_sorted

def genetic_algo(shape,nn_alg,generations=10):
    pop_fitness=np.array([random_population(shape)])
    gen=0
    fit_levels=dict.fromkeys(range(generations),[])
    while gen<generations:
        pop=simple_reproduce(pop_fitness[0])
        print(pop)
        pop_fitness=test_individuals(pop,nn_alg)
        print(pop_fitness)
        fit_levels[gen].append(pop_fitness)
        gen+=1
        print("NEW GENERATION:")
    return fit_levels
        
        

In [184]:
np.__version__

'1.12.1'

# TensorFlow MLP - Genetic Algorithm

In [233]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [234]:
np.shape(mnist.train.labels)

(55000, 10)

In [235]:
mnist.train.labels

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.]])

First I want to make one_hot vectors

In [236]:
labels = train.label.as_matrix()

In [237]:
len(np.unique(labels))

10

In [238]:
np.shape(labels)

(42000,)

In [240]:
ohlabels = np.zeros((42000,10))
ohlabels[np.arange(42000),labels]=1

In [241]:
ohlabels

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [242]:
images = train.drop('label',axis=1).as_matrix()

In [243]:
learning_rate = .001
training_epochs = 5 #?
batch_size = 200
display_step = training_epochs-1

n_hidden_1 = 256
n_hidden_2 = 256
n_input = 784
n_classes = 10

In [244]:
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [245]:
trainlist=train.as_matrix()

In [246]:
#Create simple MLP
def mlp(x, weights, biases):
    layer_1 = tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [247]:
weights = {
    'h1': tf.Variable(tf.random_normal([n_input,n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2,n_classes]))
}

In [248]:
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [249]:
def define_mlp_weights(l):
    n_hidden_1,n_hidden_2=l
    return {
    'h1': tf.Variable(tf.random_normal([n_input,n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2,n_classes]))
    }

def define_mlp_biases(l):
    n_hidden_1,n_hidden_2=l
    return {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
    }

def mlp_train(l):
    weights=define_mlp_weights(l)
    biases=define_mlp_biases(l)
    pred=mlp(x,weights,biases)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=pred))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    init = tf.initialize_all_variables()
    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
        sess.run(init)
        for epoch in range(training_epochs):
            avg_cost=0.
            total_batch = int(np.shape(images)[0]/batch_size)
            np.random.shuffle(trainlist)
            for i in range(total_batch):
                batch_x = trainlist[:,1:][i*batch_size:(i+1)*batch_size]
                lbl_y = trainlist[:,0][i*batch_size:(i+1)*batch_size]
                batch_y = np.zeros((batch_size,10))
                batch_y[np.arange(batch_size),lbl_y.astype('int')]=1
                _, c = sess.run([optimizer,cost],feed_dict={x:batch_x, y:batch_y})
                avg_cost += c / total_batch
            if epoch % display_step ==0:
                print("Epoch:", '%04d' % (epoch+1),"cost=", "{:.9f}".format(avg_cost))
        print("Optimization finished")
        correct_prediction = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        predict=tf.argmax(pred,1)
        pred_labels=predict.eval(feed_dict={x: test.as_matrix()})
        accuracy=accuracy.eval({x: mnist.test.images, y: mnist.test.labels})
    return accuracy

In [129]:
a=[[np.array([[248, 242],
       [250, 260],
       [257, 244],
       [262, 266],
       [255, 266]]), 0.89800006], [np.array([[248, 242],
       [250, 260],
       [257, 244],
       [262, 266],
       [255, 266]]), 0.89570004], [np.array([[248, 242],
       [250, 260],
       [257, 244],
       [262, 266],
       [255, 266]]), 0.89520007], [np.array([[248, 242],
       [250, 260],
       [257, 244],
       [262, 266],
       [255, 266]]), 0.89410013], [np.array([[248, 242],
       [250, 260],
       [257, 244],
       [262, 266],
       [255, 266]]), 0.8914001]]

In [132]:
a[0][0].shape

(5, 2)

In [214]:
genetic_algoss_dict=genetic_algo(np.array([[1,1]]),mlp_train)

[[256 128]
 [128 128]
 [192 128]
 [128   0]
 [ 64 192]]
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0001 cost= 149.354815501
Epoch: 0005 cost= 12.849312058
Optimization finished
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0001 cost= 115.742957201
Epoch: 0005 cost= 9.659891692
Optimization finished
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0001 cost= 140.495910763
Epoch: 0005 cost= 10.629293574
Optimization finished
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0001 cost= 2.593977552
Epoch: 0005 cost= 2.312230450
Optimization finished
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0001 cost= 140.159781797
Epoch: 0005 cost= 11.335627532
Optimization finished
[[array([256, 128]), 0.92280006], [array([192, 128]), 0.91540015], [array([128, 128]), 0.91040009], [array([ 64, 192]), 0.87550014], [array([128,   0]), 

Epoch: 0001 cost= 241.510025692
Epoch: 0005 cost= 17.223160299
Optimization finished
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0001 cost= 194.466384443
Epoch: 0005 cost= 15.969592578
Optimization finished
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0001 cost= 222.161565585
Epoch: 0005 cost= 15.776182605
Optimization finished
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0001 cost= 187.866199216
Epoch: 0005 cost= 13.795136972
Optimization finished
[[array([384, 256]), 0.94240016], [array([320, 320]), 0.94190013], [array([320, 192]), 0.94000006], [array([320, 256]), 0.93910015], [array([256, 256]), 0.93160009]]
NEW GENERATION:
[[320 256]
 [384 256]
 [384 192]
 [320 128]
 [448 256]]
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0001 cost= 220.881267420
Epoch: 0005 cost= 16.414761923
Optimization finished
Instructions for updating:
Use `tf.global

Epoch: 0001 cost= 223.190893632
Epoch: 0005 cost= 16.480914658
Optimization finished
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0001 cost= 256.730752981
Epoch: 0005 cost= 17.347432159
Optimization finished
[[array([512, 320]), 0.95370007], [array([448, 384]), 0.95130008], [array([448, 320]), 0.94580013], [array([512, 256]), 0.94480014], [array([384, 320]), 0.94380021]]
NEW GENERATION:
[[512 320]
 [512 384]
 [576 320]
 [448 320]
 [576 192]]
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0001 cost= 302.401589112
Epoch: 0005 cost= 17.531202452
Optimization finished
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0001 cost= 273.524985413
Epoch: 0005 cost= 18.704649718
Optimization finished
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0001 cost= 254.666154598
Epoch: 0005 cost= 15.089648181
Optimization finished
Instructions for updating:
Use `tf.global

In [215]:
genetic_algoss_dict[0]

[[[array([256, 128]), 0.92280006],
  [array([192, 128]), 0.91540015],
  [array([128, 128]), 0.91040009],
  [array([ 64, 192]), 0.87550014],
  [array([128,   0]), 0.097399995]],
 [[array([256, 192]), 0.93890011],
  [array([256, 128]), 0.92990005],
  [array([320, 128]), 0.92640013],
  [array([192, 192]), 0.92400014],
  [array([256,  64]), 0.91640007]],
 [[array([320, 192]), 0.94080013],
  [array([256, 256]), 0.93670011],
  [array([256, 192]), 0.93180007],
  [array([192, 192]), 0.92560011],
  [array([320,  64]), 0.89880008]],
 [[array([320, 256]), 0.9435001],
  [array([320, 192]), 0.93600011],
  [array([256, 256]), 0.93470007],
  [array([256, 192]), 0.93400007],
  [array([320, 128]), 0.92740011]],
 [[array([384, 256]), 0.94240016],
  [array([320, 320]), 0.94190013],
  [array([320, 192]), 0.94000006],
  [array([320, 256]), 0.93910015],
  [array([256, 256]), 0.93160009]],
 [[array([384, 256]), 0.94590014],
  [array([448, 256]), 0.9449001],
  [array([384, 192]), 0.93940008],
  [array([320, 2

In [41]:
pred = mlp(x, weights, biases)

In [45]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=pred))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [46]:
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [ ]:
#Modify below code to work with my dataset
#Get it to predict

In [49]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        avg_cost=0.
        total_batch = int(np.shape(images)[0]/batch_size)
        np.random.shuffle(trainlist)
        for i in range(total_batch):
            batch_x = trainlist[:,1:][i*batch_size:(i+1)*batch_size]
            lbl_y = trainlist[:,0][i*batch_size:(i+1)*batch_size]
            batch_y = np.zeros((batch_size,10))
            batch_y[np.arange(batch_size),lbl_y.astype('int')]=1
            _, c = sess.run([optimizer,cost],feed_dict={x:batch_x, y:batch_y})
            avg_cost += c / total_batch
        if epoch % display_step ==0:
            print("Epoch:", '%04d' % (epoch+1),"cost=", "{:.9f}".format(avg_cost))
    print("Optimization finished")
    correct_prediction = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    predict=tf.argmax(pred,1)
    pred_labels=predict.eval(feed_dict={x: test.as_matrix()})
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch: 0001 cost= 195.693373934
Optimization finished
Accuracy: 0.9595


In [50]:
Results['Label']=pred_labels

In [51]:
Results[['ImageId','Label']].to_csv('NNpred.csv',index=False)

In [52]:
Results.rename(columns={'Label': 'NNpred'}, inplace=True)

Test Predictions were 96.9% correct

# TensorFlow CNN

fairly brief introduction @ http://www.kdnuggets.com/2015/04/inside-deep-learning-computer-vision-convolutional-neural-networks.html

Greater understanding of the convolutional layer in TensorFlow can be found @ http://stackoverflow.com/questions/34619177/what-does-tf-nn-conv2d-do-in-tensorflow



In [261]:
learning_rate = .001
training_epochs=10
training_iters = 2500 #?
batch_size = 50
display_step = 1


n_input = 784
n_classes = 10
dropout = .5

In [262]:
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])
#keep_prob placeholder needed b/c we're doing dropout
keep_prob = tf.placeholder("float")

In [263]:
# define convolutional layer
# The changeable values strides correspond to the the stride taken by the filter window each move
# Thus default is the filter only takes a stride of 1 each time
# This function:
    # Runs filter W over x(an image) based on "strides" parameters to produce new image
    # The new image is created by the dot product between a filter and the collection of pixels the filter currently "covers"
    # adds bias to the image
    # Gives new image to a rectified linear unit(which is an activation function of type f(x) = max(0,x)) 
    # and returns the output
def conv2d(x,W,b,strides=1):
    x = tf.nn.conv2d(x,W, strides=[1,strides,strides,1],padding="SAME")
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

In [264]:
# pooling reduces dimensionality
# ksize represents size of pooling window and strides represents how much it moves every time
# For each stop the pooling window chooses only the max pixel from what it covers, then moves on
# Thus this pooling will find the max pixel of the 4(2x2) pixels it looks at each time
def maxpool2d(x,k=2):
    return tf.nn.max_pool(x, ksize=[1,k,k,1],strides = [1,k,k,1], padding='SAME')

In [265]:
# Creating Convolutional neural net with 2 Conv. and pooling layers, and A Fully connection layer w/ dropout
def conv_net(x, weights, biases, dropout):
    #x inputed as a matrix of 1x784 pixels, want to change it to 28x28 pixels
    x=tf.reshape(x, shape=[-1,28,28,1]) # Why is this -1?
    # First Conv. layer with pooling
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1,k=2)
    # Second Conv. layer with pooling
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2,k=2)
    #Fully-Connected Layer
    fc1 = tf.reshape(conv2, [-1,weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out


In [266]:
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

In [267]:
biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [291]:
def define_cnn_weights(l):
    cnv_1,cnv_2,output_1,output_2_scale=l
    return {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([cnv_1, cnv_1, 1, output_1])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([cnv_2, cnv_2, output_1, output_1*output_2_scale])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*output_1*output_2_scale, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
    }

def define_cnn_biases(l):
    cnv_1,cnv_2,output_1,output_2_scale=l
    return {
    'bc1': tf.Variable(tf.random_normal([output_1])),
    'bc2': tf.Variable(tf.random_normal([output_1*output_2_scale])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
    }

def cnn_train(l):
    weights=define_cnn_weights(l)
    biases=define_cnn_biases(l)
    print(weights)
    print(biases)
    pred = conv_net(x,weights,biases,keep_prob)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    #cost = -tf.reduce_sum(y*tf.log(pred))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    init = tf.initialize_all_variables()
    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
        sess.run(init)
        for epoch in range(training_epochs):
            avg_cost=0
            avg_acc=0
            # Keep training until reach max iterations
            total_batch = int(np.shape(images)[0]/batch_size)
            np.random.shuffle(trainlist)
            for i in range(total_batch):
                batch_x = trainlist[:,1:][i*batch_size:(i+1)*batch_size]
                lbl_y = trainlist[:,0][i*batch_size:(i+1)*batch_size]
                batch_y = np.zeros((batch_size,10))
                batch_y[np.arange(batch_size),lbl_y.astype('int')]=1
                # Run optimization op (backprop)
                sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                               keep_prob: 1.})
                if epoch % display_step == 0:
                    # Calculate batch loss and accuracy
                    loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                                      y: batch_y,
                                                                      keep_prob: 1.})
                    avg_cost += loss / total_batch
                    avg_acc += acc / total_batch
            if epoch % display_step == 0:
                print("Iter " + str(epoch) + ", Minibatch Loss= " + \
                    "{:.6f}".format(avg_cost) + ", Training Accuracy= " + \
                    "{:.5f}".format(avg_acc))
    return avg_acc
    print("Optimization Finished!")

In [292]:
genetic_algoss_dict=genetic_algo(np.array([[1,1]]),cnn_train)

[[ 3  3 72  2]
 [ 3  3 56  2]
 [ 3  3 64  2]
 [ 3  3 72  3]
 [ 3  2 64  3]
 [ 4  3 64  3]
 [ 3  3 56  1]
 [ 4  3 64  2]
 [ 2  1 64  2]
 [ 1  3 64  2]]
{'wc1': <tf.Variable 'Variable_1557:0' shape=(3, 3, 1, 72) dtype=float32_ref>, 'wc2': <tf.Variable 'Variable_1558:0' shape=(3, 3, 72, 144) dtype=float32_ref>, 'wd1': <tf.Variable 'Variable_1559:0' shape=(7056, 1024) dtype=float32_ref>, 'out': <tf.Variable 'Variable_1560:0' shape=(1024, 10) dtype=float32_ref>}
{'bc1': <tf.Variable 'Variable_1561:0' shape=(72,) dtype=float32_ref>, 'bc2': <tf.Variable 'Variable_1562:0' shape=(144,) dtype=float32_ref>, 'bd1': <tf.Variable 'Variable_1563:0' shape=(1024,) dtype=float32_ref>, 'out': <tf.Variable 'Variable_1564:0' shape=(10,) dtype=float32_ref>}
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Iter 0, Minibatch Loss= 1532.242728, Training Accuracy= 0.89481
Iter 1, Minibatch Loss= 287.731890, Training Accuracy= 0.96564
Iter 2, Minibatch Loss= 145.811196, Training Accuracy

Iter 0, Minibatch Loss= 2699.211089, Training Accuracy= 0.89290
Iter 1, Minibatch Loss= 421.937744, Training Accuracy= 0.96636
Iter 2, Minibatch Loss= 228.860254, Training Accuracy= 0.97862
Iter 3, Minibatch Loss= 151.371959, Training Accuracy= 0.98426
Iter 4, Minibatch Loss= 85.476913, Training Accuracy= 0.98974
Iter 5, Minibatch Loss= 62.784517, Training Accuracy= 0.99133
Iter 6, Minibatch Loss= 32.953254, Training Accuracy= 0.99452
Iter 7, Minibatch Loss= 23.298236, Training Accuracy= 0.99583
Iter 8, Minibatch Loss= 17.967513, Training Accuracy= 0.99631
Iter 9, Minibatch Loss= 12.850803, Training Accuracy= 0.99669
{'wc1': <tf.Variable 'Variable_1605:0' shape=(3, 3, 1, 56) dtype=float32_ref>, 'wc2': <tf.Variable 'Variable_1606:0' shape=(3, 3, 56, 56) dtype=float32_ref>, 'wd1': <tf.Variable 'Variable_1607:0' shape=(2744, 1024) dtype=float32_ref>, 'out': <tf.Variable 'Variable_1608:0' shape=(1024, 10) dtype=float32_ref>}
{'bc1': <tf.Variable 'Variable_1609:0' shape=(56,) dtype=float32_

Iter 4, Minibatch Loss= 23.537216, Training Accuracy= 0.98219
Iter 5, Minibatch Loss= 16.685421, Training Accuracy= 0.98560
Iter 6, Minibatch Loss= 12.476575, Training Accuracy= 0.98831
Iter 7, Minibatch Loss= 9.074504, Training Accuracy= 0.99033
Iter 8, Minibatch Loss= 6.214522, Training Accuracy= 0.99224
Iter 9, Minibatch Loss= 4.219668, Training Accuracy= 0.99469
{'wc1': <tf.Variable 'Variable_1645:0' shape=(3, 3, 1, 72) dtype=float32_ref>, 'wc2': <tf.Variable 'Variable_1646:0' shape=(2, 2, 72, 144) dtype=float32_ref>, 'wd1': <tf.Variable 'Variable_1647:0' shape=(7056, 1024) dtype=float32_ref>, 'out': <tf.Variable 'Variable_1648:0' shape=(1024, 10) dtype=float32_ref>}
{'bc1': <tf.Variable 'Variable_1649:0' shape=(72,) dtype=float32_ref>, 'bc2': <tf.Variable 'Variable_1650:0' shape=(144,) dtype=float32_ref>, 'bd1': <tf.Variable 'Variable_1651:0' shape=(1024,) dtype=float32_ref>, 'out': <tf.Variable 'Variable_1652:0' shape=(10,) dtype=float32_ref>}
Instructions for updating:
Use `tf.g

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Iter 0, Minibatch Loss= 2162.998972, Training Accuracy= 0.89988
Iter 1, Minibatch Loss= 450.702773, Training Accuracy= 0.96645
Iter 2, Minibatch Loss= 271.363778, Training Accuracy= 0.97769
Iter 3, Minibatch Loss= 146.790180, Training Accuracy= 0.98481
Iter 4, Minibatch Loss= 95.195174, Training Accuracy= 0.98900
Iter 5, Minibatch Loss= 58.829994, Training Accuracy= 0.99129
Iter 6, Minibatch Loss= 32.960635, Training Accuracy= 0.99431
Iter 7, Minibatch Loss= 26.238639, Training Accuracy= 0.99555
Iter 8, Minibatch Loss= 20.491128, Training Accuracy= 0.99626
Iter 9, Minibatch Loss= 15.433754, Training Accuracy= 0.99683
{'wc1': <tf.Variable 'Variable_1693:0' shape=(3, 3, 1, 80) dtype=float32_ref>, 'wc2': <tf.Variable 'Variable_1694:0' shape=(4, 4, 80, 80) dtype=float32_ref>, 'wd1': <tf.Variable 'Variable_1695:0' shape=(3920, 1024) dtype=float32_ref>, 'out': <tf.Variable 'Variable_1696:0' shape=(1024, 10) dtype=float

Iter 0, Minibatch Loss= 1823.181111, Training Accuracy= 0.89350
Iter 1, Minibatch Loss= 331.512413, Training Accuracy= 0.96650
Iter 2, Minibatch Loss= 212.184380, Training Accuracy= 0.97700
Iter 3, Minibatch Loss= 121.331757, Training Accuracy= 0.98517
Iter 4, Minibatch Loss= 71.917388, Training Accuracy= 0.98938
Iter 5, Minibatch Loss= 46.502496, Training Accuracy= 0.99236
Iter 6, Minibatch Loss= 34.149727, Training Accuracy= 0.99402
Iter 7, Minibatch Loss= 20.166857, Training Accuracy= 0.99605
Iter 8, Minibatch Loss= 15.243087, Training Accuracy= 0.99657
Iter 9, Minibatch Loss= 12.755933, Training Accuracy= 0.99674
{'wc1': <tf.Variable 'Variable_1733:0' shape=(3, 3, 1, 72) dtype=float32_ref>, 'wc2': <tf.Variable 'Variable_1734:0' shape=(2, 2, 72, 144) dtype=float32_ref>, 'wd1': <tf.Variable 'Variable_1735:0' shape=(7056, 1024) dtype=float32_ref>, 'out': <tf.Variable 'Variable_1736:0' shape=(1024, 10) dtype=float32_ref>}
{'bc1': <tf.Variable 'Variable_1737:0' shape=(72,) dtype=float32

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Iter 0, Minibatch Loss= 1859.207571, Training Accuracy= 0.87162
Iter 1, Minibatch Loss= 486.393562, Training Accuracy= 0.94921
Iter 2, Minibatch Loss= 272.062694, Training Accuracy= 0.96724
Iter 3, Minibatch Loss= 162.744917, Training Accuracy= 0.97864
Iter 4, Minibatch Loss= 115.674489, Training Accuracy= 0.98319
Iter 5, Minibatch Loss= 79.537665, Training Accuracy= 0.98629
Iter 6, Minibatch Loss= 46.116596, Training Accuracy= 0.98967
Iter 7, Minibatch Loss= 39.652580, Training Accuracy= 0.99114
Iter 8, Minibatch Loss= 23.275532, Training Accuracy= 0.99321
Iter 9, Minibatch Loss= 18.908068, Training Accuracy= 0.99405
{'wc1': <tf.Variable 'Variable_1781:0' shape=(3, 3, 1, 72) dtype=float32_ref>, 'wc2': <tf.Variable 'Variable_1782:0' shape=(3, 3, 72, 144) dtype=float32_ref>, 'wd1': <tf.Variable 'Variable_1783:0' shape=(7056, 1024) dtype=float32_ref>, 'out': <tf.Variable 'Variable_1784:0' shape=(1024, 10) dtype=flo

ResourceExhaustedError: OOM when allocating tensor with shape[17640,1024]
	 [[Node: Variable_1799/Assign = Assign[T=DT_FLOAT, _class=["loc:@Variable_1799"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/gpu:0"](Variable_1799, random_normal_1799)]]

Caused by op 'Variable_1799/Assign', defined at:
  File "D:\Anaconda\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Anaconda\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\Anaconda\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\Anaconda\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "D:\Anaconda\envs\tensorflow\lib\site-packages\zmq\ev

In [60]:
pred = conv_net(x,weights,biases,keep_prob)

In [62]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
#cost = -tf.reduce_sum(y*tf.log(pred))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [63]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [64]:
# Initializing the variables
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


running_index=0
trainlist=train.as_matrix()
img_vals=trainlist[:,0]
training_imgs = np.zeros((train.shape[0],n_classes))
training_imgs[np.arange(train.shape[0]),img_vals.astype('int')]=1
training_lbls=trainlist[:,1:]
def next_batch(batch_size):
    global running_index
    global training_imgs
    global training_lbls
    start=running_index
    running_index+=batch_size
    if running_index > train.shape[0]:
        perm = np.arange(train.shape[0])
        np.random.shuffle(perm)
        training_imgs=training_imgs[perm]
        training_lbls=training_lbls[perm]
        start=0
        running_index=batch_size
    end = running_index
    return training_lbls[start:end],training_imgs[start:end]
        

In [65]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        avg_cost=0
        avg_acc=0
        # Keep training until reach max iterations
        total_batch = int(np.shape(images)[0]/batch_size)
        np.random.shuffle(trainlist)
        for i in range(total_batch):
            batch_x = trainlist[:,1:][i*batch_size:(i+1)*batch_size]
            lbl_y = trainlist[:,0][i*batch_size:(i+1)*batch_size]
            batch_y = np.zeros((batch_size,10))
            batch_y[np.arange(batch_size),lbl_y.astype('int')]=1
            # Run optimization op (backprop)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                           keep_prob: dropout})
            if epoch % display_step == 0:
                # Calculate batch loss and accuracy
                loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                                  y: batch_y,
                                                                  keep_prob: 1.})
                avg_cost += loss / total_batch
                avg_acc += acc / total_batch
        if epoch % display_step == 0:
            print("Iter " + str(epoch) + ", Minibatch Loss= " + \
                "{:.6f}".format(avg_cost) + ", Training Accuracy= " + \
                "{:.5f}".format(avg_acc))
    print("Optimization Finished!")
    print("Predicting Testing Set Classes...")
    predict= tf.argmax(pred,1)
    pred_labels = np.zeros(test.shape[0])
    testmatrix=test.as_matrix()
    for j in range(int(test.shape[0]/batch_size)):
        if j % 56 == 0:
            print(str((j/(test.shape[0]/batch_size))*100) + "% Done")
        pred_labels[j*batch_size:(j+1)*batch_size] = predict.eval(feed_dict = {x: testmatrix[j*batch_size:(j+1)*batch_size],keep_prob:1.0})

Iter 0, Minibatch Loss= 11780.484097, Training Accuracy= 0.61388
Iter 1, Minibatch Loss= 2282.959498, Training Accuracy= 0.87433
Iter 2, Minibatch Loss= 1456.050982, Training Accuracy= 0.91202
Iter 3, Minibatch Loss= 1065.244949, Training Accuracy= 0.92929
Iter 4, Minibatch Loss= 813.728931, Training Accuracy= 0.93938
Iter 5, Minibatch Loss= 636.396363, Training Accuracy= 0.94674
Iter 6, Minibatch Loss= 502.320066, Training Accuracy= 0.95269
Iter 7, Minibatch Loss= 409.251078, Training Accuracy= 0.95681
Iter 8, Minibatch Loss= 334.412384, Training Accuracy= 0.96060
Iter 9, Minibatch Loss= 279.558497, Training Accuracy= 0.96326
Iter 10, Minibatch Loss= 237.491233, Training Accuracy= 0.96557
Iter 11, Minibatch Loss= 205.497062, Training Accuracy= 0.96669
Iter 12, Minibatch Loss= 175.805714, Training Accuracy= 0.96910
Iter 13, Minibatch Loss= 151.423010, Training Accuracy= 0.97012
Iter 14, Minibatch Loss= 131.164580, Training Accuracy= 0.97219
Iter 15, Minibatch Loss= 113.734446, Training

In [66]:
Results['Label']=pred_labels.astype('int')
Results[['ImageId','Label']].to_csv('CNpred.csv',index=False)
Results.rename(columns={'Label': 'CNNpred'}, inplace=True)

20 epochs; batch size of 50  = 96.586% Correct

In [116]:
train.shape

(42000, 785)